In [1]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import re
import xlrd


In [2]:
def convert_datetime(a):
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", 
                    "%Y-%m-%d %H:%M:%S", 
                    "%m/%d/%Y %H:%M",
                    "%m-%d-%Y %H:%M", 
                    "%m/%d/%Y", 
                    "%Y-%m-%d", 
                    "%H:%M.%f",
                    "%m/%d/%Y %H:%M:%S"] 
    for fmt in date_formats:
        try:
            return datetime.strptime(str(a), fmt).strftime("%Y-%m-%d")
        except ValueError:
            pass
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(a))).strftime("%Y-%m-%d")
    except ValueError:
        pass
    if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
        return str(a)
    return  
def input_data( data_dir  ):
    data_dir = data_dir
    out_dir = '.'

    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*.xlsx'):

        df=pd.concat(pd.read_excel(filename,sheet_name=None, skiprows=0))
        list_files.append(df)

    df1 = pd.concat(list_files,   ignore_index=True)
    
    return df1
def input_dataraw( data_dir  ):
    data_dir = data_dir
    out_dir = '.'

    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*sheet1.csv'):
        list_files.append(filename)

    df1 = pd.concat(map(pd.read_csv, list_files),   ignore_index=True)
    
    return df1
def input_data_2_format(data_dir):
    lst=[]
    lst_file = glob.glob("data_dir/*False_Rock_d*")
    for f in lst_file:
        try:
            data = pd.read_csv(f,index_col=0)
            lst.append(data)
        except:
            data = pd.read_excel(f,index_col=0)
            lst.append(data)
    df1=pd.concat(lst,ignore_index=True)
    return df1
def merge_csv_files(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

    dfs = []
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path,dtype={'BPO_QA_Task ID': object,'Object_ID':object,'Queue _ID':object})
            df['Week'] = os.path.basename(file_path).replace('.csv', '')

            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
  
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)

    return merged_df
def merge_csv_files_upl(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

    dfs = []
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path)
            df['source_file'] = os.path.basename(file_path)
            df.columns = df.columns.str.lower()
            df.columns = df.columns.map(lambda x: re.sub(r'\s+', ' ', x.strip()))
            df.columns = df.columns.str.replace(' ', '_')
            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
  
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)

    return merged_df

def merge_csv_files_side_prj(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

    dfs = []
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path, dtype={'1st Task ID': object,'queue_id':object,'Object ID':object,'2nd Task ID':object})
            df['source_file'] = os.path.basename(file_path)

            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
  
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)

    return merged_df

In [3]:
def rename_csv_files(folder_path, resolve_time_column, sheet_name=None):
    files = os.listdir(folder_path)
    
    for file in files:
         if file.endswith(".csv") and "Output" not in file:
            file_path = os.path.join(folder_path, file)
            if sheet_name is None:
                df = pd.read_csv(file_path)
            else:
                df = pd.read_csv(file_path, sheet_name=sheet_name)
                
            first_title = 'Output QA Queues'
            
            df[resolve_time_column] = pd.to_datetime(df[resolve_time_column])
            
            start_date = df[resolve_time_column].min()
            end_date = df[resolve_time_column].max()
            
            first_title = re.sub(r'[\\/:"*?<>|]+', '', first_title)
            
            start_date_str = start_date.strftime("%Y-%m-%d")
            end_date_str = end_date.strftime("%Y-%m-%d")
            
            new_file_name = f"{first_title}_{start_date_str}_{end_date_str}.csv"
            new_file_path = os.path.join(folder_path, new_file_name)
            
            if os.path.exists(new_file_path):
                os.remove(new_file_path)
            
            os.rename(file_path, new_file_path)

dir = 'Sample size - QA queues'
try:
    rename_csv_files(dir,'Date')
except: 
    pass

In [4]:
# Supporting 

#Queue
queue_list = pd.read_excel('Support_dashboard.xlsx',sheet_name='Queue List',dtype={'MOD QUEUE ID':object,'QA QUEUE ID':object})
queue_list = pd.DataFrame(queue_list)
headers = ['BUSINESS','COMPOUND','CATEGORY','MOD QUEUE ID','MOD QUEUE NAME',
           'QA QUEUE ID','QA QUEUE NAME','QUEUE GROUP','STATUS','Priority','Latency','AHT (secs)','Policy_type','Classify']
queue_list.columns = headers
queue_list = queue_list.dropna(subset=['QA QUEUE NAME'])

#Week
week_list = pd.read_excel('Support_dashboard.xlsx',sheet_name='Week')

In [5]:
week_filter = {'W45','W46'} #'W22','W23','W24','W25','W26','W27','W28','W29','W30'

In [6]:
#Completion Rate
CR_rawdata = merge_csv_files('Sample size - QA queues')
CR_rawdata = CR_rawdata[['Date',	'Queue Title',	'Queue ID',	'No. of Input','No. of Output']]
CR_rawdata['Date'] = pd.to_datetime(CR_rawdata['Date'].apply(convert_datetime))
CR_rawdata = CR_rawdata.sort_values(by=['Date','Queue Title','Queue ID','No. of Output'],ascending=True)
CR_rawdata = CR_rawdata.drop_duplicates(subset=['Date','Queue Title','Queue ID'],keep='last')
CR_rawdata['Queue Title'] = CR_rawdata['Queue Title'].str.replace('  ', ' ').str.replace('QA ã\x80\x90UG-2R-GnEã\x80\x91TTS-vi-VN', 'QA [UG-2R-GnE] TTS-vi-VN')
idx = CR_rawdata.groupby(['Date', 'Queue Title', 'Queue ID'])['No. of Output'].idxmax()
CR_rawdata = CR_rawdata.loc[idx]
CR_rawdata = pd.merge(CR_rawdata, queue_list, how='left', left_on='Queue Title', right_on='QA QUEUE NAME')
CR_1 = CR_rawdata[CR_rawdata['COMPOUND'].isin(['E-UPL','G-UPL'])]
CR_1 = pd.merge(CR_1, week_list, how='left', left_on='Date', right_on='Date')
CR_2 = CR_rawdata[~CR_rawdata['COMPOUND'].isin(['E-UPL','G-UPL'])]
CR_2 = pd.merge(CR_2, week_list, how='left', left_on='Date', right_on='QA_Date')
CR = pd.concat([CR_1, CR_2])
CR['Week'] = np.where(CR['COMPOUND'].str.contains('UPL'), CR['UPL_Week'], CR['Mod_Week'])
CR = CR[['Week', 'QA_Date','BUSINESS', 'COMPOUND','MOD QUEUE NAME','QA QUEUE NAME', 'No. of Input', 'No. of Output']]
CR_Category = CR.groupby(by=['Week','BUSINESS','QA QUEUE NAME'],as_index=False).agg({'No. of Input':'sum','No. of Output':'sum'}).rename(columns={'No. of Input':'Input','No. of Output':'Output'})
CR_Category = CR_Category[CR_Category['Output']!= 0]
CR_Category['%CR'] = CR_Category['Output']/CR_Category['Input']
CR = CR.groupby(by=['Week','COMPOUND','QA QUEUE NAME'],as_index=False).agg({'No. of Input':'sum','No. of Output':'sum'}).rename(columns={'No. of Input':'Input','No. of Output':'Output'})
CR = CR[CR['Output']!= 0]
CR = CR[CR['Week'].str.contains('|'.join(week_filter))]
CR_Category = CR_Category[CR_Category['Week'].str.contains('|'.join(week_filter))]

In [7]:
#Diff case
regular_appeal_Rawdata = merge_csv_files('Regular_appeal')
regular_appeal_Rawdata['Final Decision'] = regular_appeal_Rawdata['Final Decision'].replace({'Mods Wrong':'Failed appeal','Borderline':'Edge case','QAs Wrong':'Appeal successfully'})
BPO_QA_Decision_keep_values = ['Failed appeal', 'Appeal successfully', 'Edge case',np.nan]
temp_col = regular_appeal_Rawdata['Final Decision'].copy()
temp_col[~temp_col.isin(BPO_QA_Decision_keep_values)] = 'Failed appeal'
regular_appeal_Rawdata['Final Decision'] = temp_col
regular_appeal_Rawdata['BPO QA Date'] = pd.to_datetime(regular_appeal_Rawdata['BPO QA Date'].apply(convert_datetime))

regular_appeal_Rawdata['Final Decision'] = regular_appeal_Rawdata['Final Decision'].fillna('Mod correct')

aliases_final_decision = {"Edge case":['borderline','edge case'] ,
           "Appeal successfully":[ 'qa wrong','qas wrong', 'appeal successfully'],
           "Failed appeal":[ 'failed appeal','mod wrong','mods wrong']}

for k, v in aliases_final_decision.items():
    pat = '|'.join(v)
    regular_appeal_Rawdata.loc[regular_appeal_Rawdata['Final Decision'].str.contains(pat, case=False, na=False),'Final Decision'] = k

regular_appeal_Rawdata = regular_appeal_Rawdata.loc[:, ~regular_appeal_Rawdata.columns.str.contains('^Unnamed')]

regular_appeal_Rawdata[['Week','BPO QA Date','Final Decision']].sort_values(by=['BPO QA Date'],ascending=True)

regular_appeal_Rawdata['Final Decision'].unique()

array(['Failed appeal', 'Appeal successfully', 'Edge case', 'Mod correct'],
      dtype=object)

In [8]:
Diff_case = regular_appeal_Rawdata[['Week','Queue Name',	'mod','Final Decision','mod_date',	'BPO QA Date',	'Final Decision']]
Diff_case[Diff_case['Week']=='W44 (10.29 - 11.04)'][['Queue Name']].value_counts()

Queue Name                                 
[QA] VN LL Anchor Video 02                     32
QA VN LL Product Initial Review                32
[QA] VN LL Live 02                             15
[QA] VN LL Product Counterfeit 02              14
QA VN LL Live Counterfeit Key Frame            14
QA VN LL Anchor Video Counterfeit Key Frame    11
[QA] VN CB Product High VV 02                  10
QA VN Product IPR initial review                9
[QA]VN LL Seller On Boarding                    8
[QA] VN LL Anchor Video High VV 02              8
QA VN LL Product Buffer                         8
QA VN LL Product Report 02                      7
QA VN VAT Number Check                          7
[QA] VN LL Live Report 02                       7
QA [VN]Live Slice Appeal Audit                  6
[QA] VN LL Anchor Video Report 02               6
QA VN LL Trademark New                          6
QA VN LL Shop Decoration Risk Review            5
[QA] VN LL Product High VV 02                   5
QA VN 

In [9]:
Diff_case = regular_appeal_Rawdata[['Week','Queue Name',	'mod','mod_date',	'BPO QA Date',	'Final Decision']]
Diff_case_failed = Diff_case[(Diff_case['Final Decision']=="Failed appeal") | (Diff_case['Final Decision']=="Edge case")]
Diff_case_failed['Final Decision'].unique()

array(['Failed appeal', 'Edge case'], dtype=object)

In [10]:
# Regular/Appeal - Queues (Mod Accr)

Mod_Accr = Diff_case_failed.groupby(by=['Week', 'Queue Name'], as_index=False).agg({
    'Final Decision': [
        (' Edge case', lambda x: (x == 'Edge case').sum()), 
        (' Failed appeal', lambda x: (x == 'Failed appeal').sum())
    ]
})

Mod_Accr.columns = [''.join(col).strip() for col in Mod_Accr.columns.values]

Mod_Accr = pd.merge(CR,Mod_Accr,how='left',left_on=['Week','QA QUEUE NAME'],right_on=['Week','Queue Name'])

Mod_Accr = Mod_Accr.fillna(0)

Final_Mod_Accr = pd.merge(Mod_Accr,queue_list,how='left',left_on=['COMPOUND','QA QUEUE NAME'],right_on=['COMPOUND','QA QUEUE NAME'])

Final_Mod_Accr = Final_Mod_Accr[['Week','COMPOUND','MOD QUEUE NAME','QA QUEUE NAME','Output','Final Decision Edge case','Final Decision Failed appeal']]

Final_Mod_Accr['%Accr'] = round(1 - (Final_Mod_Accr['Final Decision Failed appeal'] / Final_Mod_Accr['Output']),4)
Final_Mod_Accr['%Overall_Accr'] = round(1 - ((Final_Mod_Accr['Final Decision Edge case']+Final_Mod_Accr['Final Decision Failed appeal']) / Final_Mod_Accr['Output']),4)
Final_Mod_Accr = Final_Mod_Accr.dropna(subset=['%Accr'])
Final_Mod_Accr = Final_Mod_Accr[Final_Mod_Accr['COMPOUND'].isin(['Anchor Video','Livestream','LL Product','CB Product','IPR','Appeal','Seller'])]
Final_Mod_Accr = Final_Mod_Accr[Final_Mod_Accr['Week'].str.contains('|'.join(week_filter))]

In [11]:
#QA ACCR

qa_dtset = merge_csv_files('QA_Diff_Cases')
qa_dtset['2nd Round Date'] = pd.to_datetime(qa_dtset['2nd Round Date'].apply(convert_datetime))
qa_dtset['2nd Round Date'].sort_values(ascending=True)

qa_dtset['1st round Name'] = qa_dtset['1st round Name'].replace('robot_estimate@','',regex=True)

qa_dtset

,Queue Name,Task ID,1st round Name,1st round Date,Object ID,1st round Result,1st round Rejection Reason,AHT,2nd round name,2nd Round Date,...,RCA 3,[BD] Comments,Screenshot,Policy screenshot,Week,Unnamed: 13,Appeal Y/N?,Appeal Remark,Final Result,Tagging
0,Inhouse QA VN LL Product Initial Review,7.15E+18,quynh.hc@trans-cosmos.com.vn,1/4/2023 17:58,7180000000000000000.0,Disapproval,"{""images:rejectReasons"":[[""Redirect_Traffic000...",18.0,zoe.nguyen,2023-01-04,...,NaN,NaN,NaN,NaN,W1 (12.31 - 01.06),NaN,NaN,NaN,NaN,NaN
1,Inhouse QA VN LL Product Initial Review,7.15E+18,quynh.hc@trans-cosmos.com.vn,1/5/2023 10:30,7180000000000000000.0,Approval,{},26.0,zoe.nguyen,2023-01-05,...,NaN,NaN,NaN,NaN,W1 (12.31 - 01.06),NaN,NaN,NaN,NaN,NaN
2,Inhouse QA VN LL Product Initial Review,7.15E+18,quynh.hc@trans-cosmos.com.vn,1/5/2023 10:31,7180000000000000000.0,Disapproval,"{""description:rejectReasons"":[[""Redirect_Traff...",21.0,zoe.nguyen,2023-01-05,...,NaN,NaN,NaN,NaN,W1 (12.31 - 01.06),NaN,NaN,NaN,NaN,NaN
3,Inhouse QA VN LL Product Initial Review,7.15E+18,quynh.hc@trans-cosmos.com.vn,1/5/2023 10:32,7180000000000000000.0,Approval,{},9.0,zoe.nguyen,2023-01-05,...,NaN,NaN,NaN,NaN,W1 (12.31 - 01.06),NaN,NaN,NaN,NaN,NaN
4,Inhouse QA VN LL Product Initial Review,7.15E+18,quynh.hc@trans-cosmos.com.vn,1/5/2023 10:32,7180000000000000000.0,Approval,{},9.0,zoe.nguyen,2023-01-05,...,NaN,NaN,NaN,NaN,W1 (12.31 - 01.06),NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,W9 (02.25 - 03.03),True,NaN,NaN,NaN,NaN
39603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,W9 (02.25 - 03.03),True,NaN,NaN,NaN,NaN
39604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,W9 (02.25 - 03.03),True,NaN,NaN,NaN,NaN
39605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,W9 (02.25 - 03.03),True,NaN,NaN,NaN,NaN


In [12]:
#QA ACCR

qa_dtset = merge_csv_files('QA_Diff_Cases')
qa_dtset['2nd Round Date'] = pd.to_datetime(qa_dtset['2nd Round Date'].apply(convert_datetime))
qa_dtset['2nd Round Date'].sort_values(ascending=True)
qa_dtset['1st round Name'] = qa_dtset['1st round Name'].replace('robot_estimate@','',regex=True)
qa_dtset = qa_dtset[['Week','2nd Round Date','Queue Name','1st round Name','Object ID','BD QA Decision']]

qa_correct = qa_dtset.groupby(['Week','1st round Name'],as_index=False)['BD QA Decision'].apply(lambda x: x[(x == 'QA Correct') | (x == 'Appeal successfully') | (x == 'Edge case') | (x.isna())].count())
qa_sampling = qa_dtset.groupby(['Week','1st round Name'],as_index=False)['Object ID'].apply(lambda y: y[(y != '')].count())
final_qa_accr = pd.merge(qa_sampling,qa_correct,how='inner',on=['Week','1st round Name']).rename(columns={'Object ID':'Sampling','BD QA Decision':'QA Correct'})
final_qa_accr['%Accr'] = round(final_qa_accr['QA Correct'] / final_qa_accr['Sampling'],4)
final_qa_accr = final_qa_accr[final_qa_accr['Week'].str.contains('|'.join(week_filter))]
final_qa_accr

,Week,1st round Name,Sampling,QA Correct,%Accr


In [13]:
# #General UPL Diff case
# G_UPL_rawdata = merge_csv_files_upl('G_UPL')
# G_UPL_rawdata = G_UPL_rawdata.dropna(subset='Queue Name')

# G_UPL_rawdata['1st Date'] = pd.to_datetime(G_UPL_rawdata['1st Date'].apply(convert_datetime))            
# G_UPL_rawdata['2nd Date'] = pd.to_datetime(G_UPL_rawdata['2nd Date'].apply(convert_datetime))
# G_UPL_rawdata = pd.merge(G_UPL_rawdata,week_list,how='left',left_on='2nd Date',right_on='Date').rename(columns={'UPL_Week':'Week'})
# G_UPL_rawdata = G_UPL_rawdata.sort_values(by=['2nd Date'],ascending=True)

# G_UPL_rawdata['BD QA Decision'] = G_UPL_rawdata['BD QA Decision'].fillna('')
# for k, v in aliases_final_decision.items():
#     pat = '|'.join(v)
#     G_UPL_rawdata.loc[G_UPL_rawdata['BD QA Decision'].str.contains(pat, case=False),'Final Decision'] = k

# G_UPL_false_case = G_UPL_rawdata[(G_UPL_rawdata['Final Decision']=='Failed appeal') | (G_UPL_rawdata['Final Decision']=='Edge case')]

# G_UPL_false_case = G_UPL_false_case.groupby(by=['Week', 'Queue Name'], as_index=False).agg({
#     'Final Decision': [
#         (' Edge case', lambda x: (x == 'Edge case').sum()), 
#         (' Failed appeal', lambda x: (x == 'Failed appeal').sum())
#     ]
# })

# G_UPL_false_case.columns = [''.join(col).strip() for col in G_UPL_false_case.columns.values]
# Final_G_UPL = pd.merge(CR,G_UPL_false_case,how='left',left_on=['Week','QA QUEUE NAME'],right_on=['Week','Queue Name'])

# Final_G_UPL = Final_G_UPL.fillna(0)

# Final_G_UPL = pd.merge(Final_G_UPL,queue_list,how='left',left_on=['COMPOUND','QA QUEUE NAME'],right_on=['COMPOUND','QA QUEUE NAME'])

# Final_G_UPL = Final_G_UPL[['Week','COMPOUND','MOD QUEUE NAME','QA QUEUE NAME','Output','Final Decision Edge case','Final Decision Failed appeal']]

# Final_G_UPL['%Accr'] = round(1 - (Final_G_UPL['Final Decision Failed appeal'] / Final_G_UPL['Output']),4)
# Final_G_UPL['%Overall_Accr'] = round(1 - ((Final_G_UPL['Final Decision Edge case']+Final_G_UPL['Final Decision Failed appeal']) / Final_G_UPL['Output']),4)
# Final_G_UPL = Final_G_UPL.dropna(subset=['%Accr'])
# Final_G_UPL = Final_G_UPL[Final_G_UPL['COMPOUND'].isin(['G-UPL'])]
# Final_G_UPL = Final_G_UPL[Final_G_UPL['Week'].str.contains('|'.join(week_filter))]

In [14]:
#Shoping Mall UPL Diff case
E_UPL_rawdata = merge_csv_files_upl('E_UPL')
E_UPL_rawdata['1st_date'] = pd.to_datetime(E_UPL_rawdata['1st_date'].apply(convert_datetime))            
E_UPL_rawdata['2nd_date'] = pd.to_datetime(E_UPL_rawdata['2nd_date'].apply(convert_datetime))
E_UPL_rawdata = pd.merge(E_UPL_rawdata,week_list,how='left',left_on='2nd_date',right_on='Date').rename(columns={'UPL_Week':'Week'})
E_UPL_rawdata['Week'].unique()

array(['W10 (03.06 - 03.12)', 'W11 (03.13 - 03.19)',
       'W12 (03.20 - 03.26)', 'W13 (03.27 - 04.02)',
       'W14 (04.03 - 04.09)', 'W15 (04.10 - 04.16)',
       'W16 (04.17 - 04.23)', 'W17 (04.24 - 04.30)',
       'W18 (05.01 - 05.07)', 'W19 (05.08 - 05.14)',
       'W20 (05.15 - 05.21)', 'W21 (05.22 - 05.28)',
       'W22 (05.29 - 06.04)', 'W23 (06.05 - 06.11)',
       'W24 (06.12 - 06.18)', 'W25 (06.19 - 06.25)',
       'W26 (06.26 - 07.02)', 'W27 (07.03 - 07.09)',
       'W28 (07.10 - 07.16)', 'W29 (07.17 - 07.23)',
       'W30 (07.24 - 07.30)', 'W31 (07.31 - 08.06)',
       'W32 (08.07 - 08.13)', 'W33 (08.14 - 08.20)',
       'W34 (08.21 - 08.27)', 'W35 (08.28 - 09.03)',
       'W36 (09.04 - 09.10)', 'W37 (09.11 - 09.17)',
       'W38 (09.19 - 09.25)', 'W39 (09.29 - 10.04)', nan,
       'W40 (10.05 - 10.11)', 'W41 (10.12 - 10.18)',
       'W42 (10.19 - 10.25)', 'W43 (10.26 - 11.01)',
       'W44 (11.02 - 11.08)', 'W45 (11.09 - 11.15)',
       'W46 (11.16 - 11.22)', 'W47 (11.23

In [15]:
#Shoping Mall UPL Diff case
E_UPL_rawdata = merge_csv_files_upl('E_UPL')
E_UPL_rawdata['1st_date'] = pd.to_datetime(E_UPL_rawdata['1st_date'].apply(convert_datetime))            
E_UPL_rawdata['2nd_date'] = pd.to_datetime(E_UPL_rawdata['2nd_date'].apply(convert_datetime))
E_UPL_rawdata = pd.merge(E_UPL_rawdata,week_list,how='left',left_on='2nd_date',right_on='Date').rename(columns={'UPL_Week':'Week'})
E_UPL_rawdata = E_UPL_rawdata[~E_UPL_rawdata['Week'].isnull()]
E_UPL_rawdata.sort_values(by=['2nd_date'],ascending=True)
E_UPL_rawdata['[bpo_qa]_decision'] = E_UPL_rawdata['[bpo_qa]_decision'].fillna('')
for k, v in aliases_final_decision.items():
    pat = '|'.join(v)
    E_UPL_rawdata.loc[E_UPL_rawdata['[bpo_qa]_decision'].str.contains(pat, case=False),'final_result'] = k
E_UPL_rawdata_1 = E_UPL_rawdata.copy()
E_UPL_rawdata_1['final_result_(copy)'] = E_UPL_rawdata_1['final_result']
E_UPL_false_case = E_UPL_rawdata_1.groupby(by=['Week','queue_name'], as_index=False)\
                         .agg({'final_result': lambda x: x[x.eq('Failed appeal')].count() if 'Failed appeal' in x.values else 0,
                               'final_result_(copy)': lambda x: x[x.eq('Edge case')].count() if 'Edge case' in x.values else 0})\
                         .rename(columns={'queue_name': 'Queue Name', 'final_result': 'Final Decision Failed appeal', 'final_result_(copy)':'Final Decision Edge case'})
E_UPL_false_case = E_UPL_false_case[['Week','Queue Name','Final Decision Edge case','Final Decision Failed appeal']]
Final_E_UPL = pd.merge(CR,E_UPL_false_case,how='left',left_on=['Week','QA QUEUE NAME'],right_on=['Week','Queue Name'])
Final_E_UPL = Final_E_UPL.fillna(0)
Final_E_UPL = pd.merge(Final_E_UPL,queue_list,how='left',left_on=['COMPOUND','QA QUEUE NAME'],right_on=['COMPOUND','QA QUEUE NAME'])
Final_E_UPL = Final_E_UPL[Final_E_UPL['COMPOUND']=="E-UPL"][['Week','COMPOUND','MOD QUEUE NAME','QA QUEUE NAME','Output','Final Decision Edge case','Final Decision Failed appeal']]
Final_E_UPL = Final_E_UPL.groupby(by=['Week','COMPOUND','MOD QUEUE NAME','QA QUEUE NAME'],as_index=False).agg({'Output':'sum','Final Decision Edge case':'sum','Final Decision Failed appeal':'sum'})
Final_E_UPL = Final_E_UPL[['Week','COMPOUND','MOD QUEUE NAME','QA QUEUE NAME','Output','Final Decision Edge case','Final Decision Failed appeal']]
Final_E_UPL['%Accr'] = 1-(Final_E_UPL['Final Decision Failed appeal'] / Final_E_UPL['Output'])
Final_E_UPL['%Overall_Accr'] = 1-((Final_E_UPL['Final Decision Edge case']+Final_E_UPL['Final Decision Failed appeal']) / Final_E_UPL['Output'])
Final_E_UPL = Final_E_UPL.dropna(subset=['%Accr'])
Final_E_UPL = Final_E_UPL[Final_E_UPL['Week'].str.contains('|'.join(week_filter))]
Final_E_UPL

C:\Users\v6210227\AppData\Local\Temp\ipykernel_10012\1047983913.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Edge case' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  E_UPL_rawdata.loc[E_UPL_rawdata['[bpo_qa]_decision'].str.contains(pat, case=False),'final_result'] = k


,Week,COMPOUND,MOD QUEUE NAME,QA QUEUE NAME,Output,Final Decision Edge case,Final Decision Failed appeal,%Accr,%Overall_Accr
0,W45 (11.09 - 11.15),E-UPL,[VN] Mall UP Recommendation CB Sample,[VN] Mall UP Recommendation CB Sample QA,44,0.0,1.0,0.977273,0.977273
1,W45 (11.09 - 11.15),E-UPL,[VN] Mall UP Recommendation L2L Sample,[VN] Mall UP Recommendation L2L Sample QA,230,0.0,1.0,0.995652,0.995652
2,W45 (11.09 - 11.15),E-UPL,[VN] Mall UP Search CB Sample,[VN] Mall UP Search CB Sample QA,23,0.0,0.0,1.000000,1.000000
3,W45 (11.09 - 11.15),E-UPL,[VN] Mall UP Search L2L Sample,[VN] Mall UP Search L2L Sample QA,302,0.0,1.0,0.996689,0.996689
4,W46 (11.16 - 11.22),E-UPL,[VN] Mall UP Recommendation CB Sample,[VN] Mall UP Recommendation CB Sample QA,21,0.0,0.0,1.000000,1.000000
5,W46 (11.16 - 11.22),E-UPL,[VN] Mall UP Recommendation L2L Sample,[VN] Mall UP Recommendation L2L Sample QA,263,0.0,0.0,1.000000,1.000000
6,W46 (11.16 - 11.22),E-UPL,[VN] Mall UP Search CB Sample,[VN] Mall UP Search CB Sample QA,23,0.0,0.0,1.000000,1.000000
7,W46 (11.16 - 11.22),E-UPL,[VN] Mall UP Search L2L Sample,[VN] Mall UP Search L2L Sample QA,297,0.0,2.0,0.993266,0.993266


In [16]:
#Side project
Sideproject_rawdata = merge_csv_files_side_prj('Side_project')
Sideproject_rawdata['BPO_QA Date'] = pd.to_datetime(Sideproject_rawdata['BPO QA Date'].apply(convert_datetime))
Sideproject_rawdata = Sideproject_rawdata.loc[:, ~Sideproject_rawdata.columns.str.contains('^Unnamed')]
Sideproject_rawdata['Queue Name'] = Sideproject_rawdata['Queue Name'].str.replace('QA ?UG-2R-GnE?TTS-vi-VN', 'QA [UG-2R-GnE] TTS-vi-VN')
Sideproject_rawdata = pd.merge(Sideproject_rawdata,week_list,how='left',left_on='BPO_QA Date',right_on='QA_Date').rename(columns={'Mod_Week':'Week'})
Sideproject_rawdata = Sideproject_rawdata.sort_values(by=['BPO QA Date'],ascending=True)
Sideproject_rawdata = Sideproject_rawdata.dropna(subset=['Queue Name'])

Sideproject_rawdata = Sideproject_rawdata.groupby(['Week','Queue Name'], as_index=False).apply(lambda x: pd.Series({
    'Final Decision Edge case': (x['Final Decision'] == 'Edge case').sum(),
    'Final Decision Failed appeal': (x['Final Decision'] == 'Failed appeal').sum()
}))
Final_Mod_Accr_sideproject = pd.merge(CR,Sideproject_rawdata,how='left',left_on=['Week','QA QUEUE NAME'],right_on=['Week','Queue Name'])
Final_Mod_Accr_sideproject = Final_Mod_Accr_sideproject[Final_Mod_Accr_sideproject['COMPOUND']=='Side Project']
Final_Mod_Accr_sideproject = Final_Mod_Accr_sideproject.fillna(0)
Final_Mod_Accr_sideproject = pd.merge(Final_Mod_Accr_sideproject,queue_list,how='left',left_on=['COMPOUND','QA QUEUE NAME'],right_on=['COMPOUND','QA QUEUE NAME'])
Final_Mod_Accr_sideproject = Final_Mod_Accr_sideproject[['Week','COMPOUND','MOD QUEUE NAME','QA QUEUE NAME','Output','Final Decision Edge case','Final Decision Failed appeal']]
Final_Mod_Accr_sideproject['%Accr'] = 1-(Final_Mod_Accr_sideproject['Final Decision Failed appeal'] / Final_Mod_Accr_sideproject['Output'])
Final_Mod_Accr_sideproject['%Overall_Accr'] = 1-((Final_Mod_Accr_sideproject['Final Decision Edge case']+Final_Mod_Accr_sideproject['Final Decision Failed appeal']) / Final_Mod_Accr_sideproject['Output'])
Final_Mod_Accr_sideproject = Final_Mod_Accr_sideproject.dropna(subset=['%Accr'])
Final_Mod_Accr_sideproject = Final_Mod_Accr_sideproject[Final_Mod_Accr_sideproject['Week'].str.contains('|'.join(week_filter))]
Final_Mod_Accr_sideproject

,Week,COMPOUND,MOD QUEUE NAME,QA QUEUE NAME,Output,Final Decision Edge case,Final Decision Failed appeal,%Accr,%Overall_Accr
0,W45 (11.05 - 11.11),Side Project,[CB][VN]-Shoptab Product Image Dedicated,QA [CB][VN]-Shoptab Product Image Dedicated,399,1.0,1.0,0.997494,0.994987
1,W45 (11.05 - 11.11),Side Project,[L2L][VN]-Shoptab Product Image Dedicated,QA [L2L][VN]-Shoptab Product Image Dedicated,399,0.0,0.0,1.000000,1.000000
2,W45 (11.05 - 11.11),Side Project,[VN] Seller Gov X Feige Chat,QA [VN] Seller Gov X Feige Chat,114,1.0,2.0,0.982456,0.973684
3,W45 (11.05 - 11.11),Side Project,[VN] Shoptab Content cover image,QA [VN] Shoptab Content cover image,630,0.0,0.0,1.000000,1.000000
4,W46 (11.12 - 11.18),Side Project,[CB][VN]-Shoptab Product Image Dedicated,QA [CB][VN]-Shoptab Product Image Dedicated,399,0.0,2.0,0.994987,0.994987
5,W46 (11.12 - 11.18),Side Project,[L2L][VN]-Shoptab Product Image Dedicated,QA [L2L][VN]-Shoptab Product Image Dedicated,399,0.0,0.0,1.000000,1.000000
6,W46 (11.12 - 11.18),Side Project,[VN] Seller Gov X Feige Chat,QA [VN] Seller Gov X Feige Chat,191,0.0,2.0,0.989529,0.989529
7,W46 (11.12 - 11.18),Side Project,[VN] Shoptab Content cover image,QA [VN] Shoptab Content cover image,715,0.0,0.0,1.000000,1.000000


In [17]:
mod_performance = pd.concat([Final_Mod_Accr,Final_Mod_Accr_sideproject,Final_E_UPL],axis=0)
mod_performance = mod_performance.sort_values(by='Week', ascending=False)
mod_performance[['COMPOUND','Week']].value_counts()

COMPOUND      Week               
Appeal        W45 (11.05 - 11.11)    9
              W46 (11.12 - 11.18)    9
LL Product    W46 (11.12 - 11.18)    8
              W45 (11.05 - 11.11)    8
IPR           W46 (11.12 - 11.18)    6
              W45 (11.05 - 11.11)    6
Anchor Video  W46 (11.12 - 11.18)    5
Seller        W46 (11.12 - 11.18)    5
              W45 (11.05 - 11.11)    5
Anchor Video  W45 (11.05 - 11.11)    5
CB Product    W46 (11.12 - 11.18)    5
              W45 (11.05 - 11.11)    5
E-UPL         W46 (11.16 - 11.22)    4
              W45 (11.09 - 11.15)    4
Livestream    W45 (11.05 - 11.11)    4
              W46 (11.12 - 11.18)    4
Side Project  W45 (11.05 - 11.11)    4
              W46 (11.12 - 11.18)    4
Name: count, dtype: int64

In [18]:
all_sheets_WBR = {'Completion Rate': CR_Category,
                  'Mod performance':mod_performance,
                  'QA Accuracy': final_qa_accr}

In [19]:
mod_performance.sort_values(by='COMPOUND',ascending=True)

,Week,COMPOUND,MOD QUEUE NAME,QA QUEUE NAME,Output,Final Decision Edge case,Final Decision Failed appeal,%Accr,%Overall_Accr
57,W46 (11.12 - 11.18),Anchor Video,VN LL No-Anchor Video,QA VN LL No-Anchor Video,273,0.0,0.0,1.000000,1.000000
1,W45 (11.05 - 11.11),Anchor Video,VN LL No-Anchor Video,QA VN LL No-Anchor Video,273,0.0,0.0,1.000000,1.000000
3,W45 (11.05 - 11.11),Anchor Video,VN LL Anchor Video High VV 02,[QA] VN LL Anchor Video High VV 02,684,1.0,4.0,0.994200,0.992700
4,W45 (11.05 - 11.11),Anchor Video,VN LL Anchor Video Report 02,[QA] VN LL Anchor Video Report 02,684,1.0,9.0,0.986800,0.985400
2,W45 (11.05 - 11.11),Anchor Video,VN LL Anchor Video 02,[QA] VN LL Anchor Video 02,1770,1.0,25.0,0.985900,0.985300
...,...,...,...,...,...,...,...,...,...
7,W46 (11.12 - 11.18),Side Project,[VN] Shoptab Content cover image,QA [VN] Shoptab Content cover image,715,0.0,0.0,1.000000,1.000000
6,W46 (11.12 - 11.18),Side Project,[VN] Seller Gov X Feige Chat,QA [VN] Seller Gov X Feige Chat,191,0.0,2.0,0.989529,0.989529
5,W46 (11.12 - 11.18),Side Project,[L2L][VN]-Shoptab Product Image Dedicated,QA [L2L][VN]-Shoptab Product Image Dedicated,399,0.0,0.0,1.000000,1.000000
4,W46 (11.12 - 11.18),Side Project,[CB][VN]-Shoptab Product Image Dedicated,QA [CB][VN]-Shoptab Product Image Dedicated,399,0.0,2.0,0.994987,0.994987


In [20]:
WBR_writer = pd.ExcelWriter('WBR.xlsx', engine='xlsxwriter')

for sheet_name in all_sheets_WBR.keys():
    all_sheets_WBR[sheet_name].to_excel(WBR_writer, sheet_name=sheet_name, index=True)

WBR_writer.close()